In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx

In [2]:
# make sure pandas is version 1.0 or higher
# make sure networkx is verion 2.4 or higher
print(pd.__version__)
print(nx.__version__)

1.5.3
2.8.4


In [3]:
from ema_workbench import (
    Model,
    Policy,
    ema_logging,
    SequentialEvaluator,
    MultiprocessingEvaluator,
)
from dike_model_function import DikeNetwork  # @UnresolvedImport
from problem_formulation import get_model_for_problem_formulation, sum_over, sum_over_time



In [4]:
ema_logging.log_to_stderr(ema_logging.INFO)

# choose problem formulation number, between 0-5
# each problem formulation has its own list of outcomes
dike_model, planning_steps = get_model_for_problem_formulation(6)

In [5]:
# enlisting uncertainties, their types (RealParameter/IntegerParameter/CategoricalParameter), lower boundary, and upper boundary
import copy

for unc in dike_model.uncertainties:
    print(repr(unc))

uncertainties = copy.deepcopy(dike_model.uncertainties)

CategoricalParameter('discount rate 0', [0, 1, 2, 3])
IntegerParameter('A.0_ID flood wave shape', 0, 132, resolution=None, default=None, variable_name=['A.0_ID flood wave shape'], pff=False)
RealParameter('A.1_Bmax', 30, 350, resolution=None, default=None, variable_name=['A.1_Bmax'], pff=False)
RealParameter('A.1_pfail', 0, 1, resolution=None, default=None, variable_name=['A.1_pfail'], pff=False)
CategoricalParameter('A.1_Brate', [0, 1, 2])
RealParameter('A.2_Bmax', 30, 350, resolution=None, default=None, variable_name=['A.2_Bmax'], pff=False)
RealParameter('A.2_pfail', 0, 1, resolution=None, default=None, variable_name=['A.2_pfail'], pff=False)
CategoricalParameter('A.2_Brate', [0, 1, 2])
RealParameter('A.3_Bmax', 30, 350, resolution=None, default=None, variable_name=['A.3_Bmax'], pff=False)
RealParameter('A.3_pfail', 0, 1, resolution=None, default=None, variable_name=['A.3_pfail'], pff=False)
CategoricalParameter('A.3_Brate', [0, 1, 2])
RealParameter('A.4_Bmax', 30, 350, resolution=N

In [6]:
# enlisting policy levers, their types (RealParameter/IntegerParameter), lower boundary, and upper boundary
for policy in dike_model.levers:
    print(repr(policy))

levers = copy.deepcopy(dike_model.levers)

IntegerParameter('0_RfR 0', 0, 1, resolution=None, default=None, variable_name=['0_RfR 0'], pff=False)
IntegerParameter('1_RfR 0', 0, 1, resolution=None, default=None, variable_name=['1_RfR 0'], pff=False)
IntegerParameter('2_RfR 0', 0, 1, resolution=None, default=None, variable_name=['2_RfR 0'], pff=False)
IntegerParameter('3_RfR 0', 0, 1, resolution=None, default=None, variable_name=['3_RfR 0'], pff=False)
IntegerParameter('4_RfR 0', 0, 1, resolution=None, default=None, variable_name=['4_RfR 0'], pff=False)
IntegerParameter('EWS_DaysToThreat', 0, 4, resolution=None, default=None, variable_name=['EWS_DaysToThreat'], pff=False)
IntegerParameter('A.1_DikeIncrease 0', 0, 10, resolution=None, default=None, variable_name=['A.1_DikeIncrease 0'], pff=False)
IntegerParameter('A.2_DikeIncrease 0', 0, 10, resolution=None, default=None, variable_name=['A.2_DikeIncrease 0'], pff=False)
IntegerParameter('A.3_DikeIncrease 0', 0, 10, resolution=None, default=None, variable_name=['A.3_DikeIncrease 0'

In [7]:
# defining specific policies
# for example, policy 1 is about extra protection in upper boundary
# policy 2 is about extra protection in lower boundary
# policy 3 is extra protection in random locations


def get_do_nothing_dict():
    return {l.name: 0 for l in dike_model.levers}

policies = [
    Policy(
        "Baseline",
        **dict(
            get_do_nothing_dict(),
            **{"0_RfR 0": 0, "1_RfR 0": 0,"2_RfR 0": 0, "3_RfR 0": 0, "4_RfR 0": 0,
               "A.1_DikeIncrease 0": 0,"A.2_DikeIncrease 0": 0,"A.3_DikeIncrease 0": 0,"A.4_DikeIncrease 0": 0, "A.5_DikeIncrease 0": 0,
               'EWS_DaysToThreat':0
               }
        )
    )]

max_policies=[
    Policy(
        "DI max",
        **dict(
            get_do_nothing_dict(),
            **{"A.1_DikeIncrease 0": 10,"A.2_DikeIncrease 0": 10,"A.3_DikeIncrease 0": 10,"A.4_DikeIncrease 0": 10, "A.5_DikeIncrease 0": 10}
        )
    ),
        Policy(
        "RfR max",
        **dict(
            get_do_nothing_dict(),
            **{"0_RfR 0": 1, "1_RfR 0": 1,"2_RfR 0": 1, "3_RfR 0": 1, "4_RfR 0": 1,
               "A.1_DikeIncrease 0": 0,"A.2_DikeIncrease 0": 0,"A.3_DikeIncrease 0": 0,"A.4_DikeIncrease 0": 0, "A.5_DikeIncrease 0": 0,
               'EWS_DaysToThreat':0
               }
        )
    ),
        Policy(
        "EWS max",
        **dict(
            get_do_nothing_dict(),
            **{"0_RfR 0": 0, "1_RfR 0": 0, "2_RfR 0": 0, "3_RfR 0": 0, "4_RfR 0": 0,
               "A.1_DikeIncrease 0": 0,"A.2_DikeIncrease 0": 0,"A.3_DikeIncrease 0": 0,"A.4_DikeIncrease 0": 0, "A.5_DikeIncrease 0": 0,
               'EWS_DaysToThreat':4
               }
        )
    ),
        Policy(
        "All max",
        **dict(
            get_do_nothing_dict(),
            **{"0_RfR 0": 1, "1_RfR 0": 1, "2_RfR 0": 1, "3_RfR 0": 1, "4_RfR 0": 1,
               "A.1_DikeIncrease 0": 5,"A.2_DikeIncrease 0": 5,"A.3_DikeIncrease 0": 5,"A.4_DikeIncrease 0": 5, "A.5_DikeIncrease 0": 5,
               'EWS_DaysToThreat':4
               }
        )
    )]
    
Di_Ge_policies=[
        Policy(
        "Di-Ge RfR-Ov max",
        **dict(
            get_do_nothing_dict(),
            **{"0_RfR 0": 0, "1_RfR 0": 0,"2_RfR 0": 0, "3_RfR 0": 1, "4_RfR 0": 1,
               "A.1_DikeIncrease 0": 8,"A.2_DikeIncrease 0": 8,"A.3_DikeIncrease 0": 8,"A.4_DikeIncrease 0": 0, "A.5_DikeIncrease 0": 0,
               'EWS_DaysToThreat':0
               }
        )
    ),
        Policy(
        "DiGe RfROv mod",
        **dict(
            get_do_nothing_dict(),
            **{"0_RfR 0": 0, "1_RfR 0": 0,"2_RfR 0": 0, "3_RfR 0": 1, "4_RfR 0": 1,
               "A.1_DikeIncrease 0": 3,"A.2_DikeIncrease 0": 2,"A.3_DikeIncrease 0": 4,"A.4_DikeIncrease 0": 0, "A.5_DikeIncrease 0": 0,
               'EWS_DaysToThreat':0
               }
        )
    ),
        Policy(
        "DiGe RfROv mod&EWS",
        **dict(
            get_do_nothing_dict(),
            **{"0_RfR 0": 0, "1_RfR 0": 0,"2_RfR 0": 0, "3_RfR 0": 1, "4_RfR 0": 1,
               "A.1_DikeIncrease 0": 3,"A.2_DikeIncrease 0": 2,"A.3_DikeIncrease 0": 4,"A.4_DikeIncrease 0": 0, "A.5_DikeIncrease 0": 0,
               'EWS_DaysToThreat':2
               }
        )
    )]

infrastr_policies=[
        Policy(
        "Comb Green",
        **dict(
            get_do_nothing_dict(),
            **{"0_RfR 0": 1, "1_RfR 0": 1,"2_RfR 0": 0, "3_RfR 0": 1, "4_RfR 0": 1,
               "A.1_DikeIncrease 0": 0,"A.2_DikeIncrease 0": 0,"A.3_DikeIncrease 0": 4,"A.4_DikeIncrease 0": 0, "A.5_DikeIncrease 0": 0,
               'EWS_DaysToThreat':0
               }
        )
    ),
        Policy(
        "Comb Grey",
        **dict(
            get_do_nothing_dict(),
            **{"0_RfR 0": 0, "1_RfR 0": 1,"2_RfR 0": 0, "3_RfR 0": 1, "4_RfR 0": 0,
               "A.1_DikeIncrease 0": 3,"A.2_DikeIncrease 0": 0,"A.3_DikeIncrease 0": 4,"A.4_DikeIncrease 0": 0, "A.5_DikeIncrease 0": 4,
               'EWS_DaysToThreat':0
               }
        )
    ),
        Policy(
        "Comb GreenEWS",
        **dict(
            get_do_nothing_dict(),
            **{"0_RfR 0": 1, "1_RfR 0": 1,"2_RfR 0": 0, "3_RfR 0": 1, "4_RfR 0": 1,
               "A.1_DikeIncrease 0": 0,"A.2_DikeIncrease 0": 0,"A.3_DikeIncrease 0": 4,"A.4_DikeIncrease 0": 0, "A.5_DikeIncrease 0": 0,
               'EWS_DaysToThreat':2
               }
        ))]

policy_old = [Policy(
        "Policy 1",
        **dict(
            get_do_nothing_dict(),
            **{"0_RfR 0": 1, "0_RfR 1": 1, "0_RfR 2": 1, "A.1_DikeIncrease 0": 5}
        )
    ),
    Policy(
        "Policy 2",
        **dict(
            get_do_nothing_dict(),
            **{"4_RfR 0": 1, "4_RfR 1": 1, "4_RfR 2": 1, "A.5_DikeIncrease 0": 5}
        )
    ),
    Policy(
        "Policy 3",
        **dict(
            get_do_nothing_dict(),
            **{"1_RfR 0": 1, "2_RfR 1": 1, "3_RfR 2": 1, "A.3_DikeIncrease 0": 5}
        )
    )]

DR12_OP_v1_manual= [
        Policy(
        "DR1&2 OP3",
        **dict(
            get_do_nothing_dict(),
            **{"0_RfR 0": 0, "1_RfR 0": 0,"2_RfR 0": 1, "3_RfR 0": 0, "4_RfR 0": 0,
               "A.1_DikeIncrease 0": 2,"A.2_DikeIncrease 0": 3,"A.3_DikeIncrease 0": 10,"A.4_DikeIncrease 0": 4, "A.5_DikeIncrease 0": 6,
               'EWS_DaysToThreat':1
               }
        )
    ),
        Policy(
        "DR1&2 OP1",
        **dict(
            get_do_nothing_dict(),
            **{"0_RfR 0": 0, "1_RfR 0": 0,"2_RfR 0": 0, "3_RfR 0": 0, "4_RfR 0": 0,
               "A.1_DikeIncrease 0": 2,"A.2_DikeIncrease 0": 4,"A.3_DikeIncrease 0": 5,"A.4_DikeIncrease 0": 9, "A.5_DikeIncrease 0": 6,
               'EWS_DaysToThreat':1
               }
        )
    ),
        Policy(
        "DR1&2 OP0",
        **dict(
            get_do_nothing_dict(),
            **{"0_RfR 0": 0, "1_RfR 0": 0,"2_RfR 0": 0, "3_RfR 0": 0, "4_RfR 0": 0,
               "A.1_DikeIncrease 0": 0,"A.2_DikeIncrease 0": 4,"A.3_DikeIncrease 0": 9,"A.4_DikeIncrease 0": 5, "A.5_DikeIncrease 0": 6,
               'EWS_DaysToThreat':1
               }
        )
    ),
        Policy(
        "DR1&2 OP2",
        **dict(
            get_do_nothing_dict(),
            **{"0_RfR 0": 0, "1_RfR 0": 0,"2_RfR 0": 1, "3_RfR 0": 0, "4_RfR 0": 0,
               "A.1_DikeIncrease 0": 0,"A.2_DikeIncrease 0": 3,"A.3_DikeIncrease 0": 5,"A.4_DikeIncrease 0": 5, "A.5_DikeIncrease 0": 6,
               'EWS_DaysToThreat':1
               }
        )
    )]

In [8]:
from funs_policy_eval import create_optimization_policies

proposed_policies_files = ["OptimizationGel1_4Pol_v1.csv","OptimizationResults_DR12_v1.csv"]


DR12_OP_v1 = create_optimization_policies("OptimizationResults_DR12_v1.csv","DR12_v1_P",dike_model)
GEL_OP_v1 = create_optimization_policies("OptimizationGel1_4Pol_v1.csv","GEL_OP_v1",dike_model)

#+Di_Ge_policies+DR12_OP_v1+infrastr_policies+

policies=policies+GEL_OP_v1

In [9]:
# pass the policies list to EMA workbench experiment runs
n_scenarios = 100
with MultiprocessingEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(n_scenarios, policies)

[MainProcess/INFO] pool started with 8 workers
[MainProcess/INFO] performing 100 scenarios * 5 policies * 1 model(s) = 500 experiments
100%|████████████████████████████████████████| 500/500 [00:23<00:00, 21.01it/s]
[MainProcess/INFO] experiments finished
[MainProcess/INFO] terminating pool


In [10]:
experiments, outcomes = results
print(experiments.keys())
experiments.info()

Index(['A.0_ID flood wave shape', 'A.1_Bmax', 'A.1_Brate', 'A.1_pfail',
       'A.2_Bmax', 'A.2_Brate', 'A.2_pfail', 'A.3_Bmax', 'A.3_Brate',
       'A.3_pfail', 'A.4_Bmax', 'A.4_Brate', 'A.4_pfail', 'A.5_Bmax',
       'A.5_Brate', 'A.5_pfail', 'discount rate 0', '0_RfR 0', '1_RfR 0',
       '2_RfR 0', '3_RfR 0', '4_RfR 0', 'EWS_DaysToThreat',
       'A.1_DikeIncrease 0', 'A.2_DikeIncrease 0', 'A.3_DikeIncrease 0',
       'A.4_DikeIncrease 0', 'A.5_DikeIncrease 0', 'scenario', 'policy',
       'model'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype   
---  ------                   --------------  -----   
 0   A.0_ID flood wave shape  500 non-null    int32   
 1   A.1_Bmax                 500 non-null    float64 
 2   A.1_Brate                500 non-null    category
 3   A.1_pfail                500 non-null    float64 
 4   A.2_Bmax                 500 

In [11]:
print(outcomes.keys())

print(outcomes["RfR Total Costs"].shape)

dict_keys(['A.1_Expected Annual Damage', 'A.1_Dike Investment Costs', 'A.1_Expected Number of Deaths', 'A.2_Expected Annual Damage', 'A.2_Dike Investment Costs', 'A.2_Expected Number of Deaths', 'A.3_Expected Annual Damage', 'A.3_Dike Investment Costs', 'A.3_Expected Number of Deaths', 'A.4_Expected Annual Damage', 'A.4_Dike Investment Costs', 'A.4_Expected Number of Deaths', 'A.5_Expected Annual Damage', 'A.5_Dike Investment Costs', 'A.5_Expected Number of Deaths', 'RfR Total Costs', 'Expected Evacuation Costs'])
(500,)


In [12]:
'''num_planning_steps=3

for key, values in outcomes.items():
    # Repeat the values for each planning step
    repeated_values = np.repeat(values, num_planning_steps, axis=0)

    # Create a planning step column
    planning_steps = np.tile(np.arange(num_planning_steps), len(values)).reshape(-1, 1)

    # Create a column index array
    column_index = np.arange(len(values[0]))

    # Repeat the column index array for each planning step
    repeated_column_index = np.tile(column_index, num_planning_steps)

    # Create a multi-index dataframe
    temp_df = pd.DataFrame(
        np.column_stack((planning_steps, repeated_column_index, repeated_values)),
        columns=['Planning Step', 'Column', key]
    )

    # Append the temporary dataframe to the main dataframe
    df = pd.concat([df, temp_df], ignore_index=True)




outcomes_pivoted = df
outcomes_pivoted.tail()'''

"num_planning_steps=3\n\nfor key, values in outcomes.items():\n    # Repeat the values for each planning step\n    repeated_values = np.repeat(values, num_planning_steps, axis=0)\n\n    # Create a planning step column\n    planning_steps = np.tile(np.arange(num_planning_steps), len(values)).reshape(-1, 1)\n\n    # Create a column index array\n    column_index = np.arange(len(values[0]))\n\n    # Repeat the column index array for each planning step\n    repeated_column_index = np.tile(column_index, num_planning_steps)\n\n    # Create a multi-index dataframe\n    temp_df = pd.DataFrame(\n        np.column_stack((planning_steps, repeated_column_index, repeated_values)),\n        columns=['Planning Step', 'Column', key]\n    )\n\n    # Append the temporary dataframe to the main dataframe\n    df = pd.concat([df, temp_df], ignore_index=True)\n\n\n\n\noutcomes_pivoted = df\noutcomes_pivoted.tail()"

In [14]:
from funs_viz import tidy_results

results_df = tidy_results(results,6,1)
results_df.to_csv("results/test_1.csv")

print(results_df.columns)
results_df.head()

Index(['A.0_ID flood wave shape', 'discount rate 0', 'EWS_DaysToThreat',
       'scenario', 'policy', 'model', 'RfR Total Costs',
       'Expected Evacuation Costs', 'Dike ring', 'Bmax', 'Brate', 'pfail',
       'RfR 0', 'DikeIncrease 0', 'Dike Investment Costs',
       'Expected Annual Damage', 'Expected Number of Deaths'],
      dtype='object')


,A.0_ID flood wave shape,discount rate 0,EWS_DaysToThreat,scenario,policy,model,RfR Total Costs,Expected Evacuation Costs,Dike ring,Bmax,Brate,pfail,RfR 0,DikeIncrease 0,Dike Investment Costs,Expected Annual Damage,Expected Number of Deaths
0,101,2.5,0,0,Baseline,dikesnet,0.0,0.0,A.1,296.145689,1.5,0.382395,0,0,0,7.535597e+08,0.454297
1,65,4.5,0,1,Baseline,dikesnet,0.0,0.0,A.1,267.775888,1.5,0.071956,0,0,0,7.141333e+08,0.675936
2,91,2.5,0,2,Baseline,dikesnet,0.0,0.0,A.1,164.507300,10.0,0.452350,0,0,0,1.851189e+08,0.115894
3,130,2.5,0,3,Baseline,dikesnet,0.0,0.0,A.1,88.354705,1.0,0.428821,0,0,0,2.115278e+08,0.130926
4,77,3.5,0,4,Baseline,dikesnet,0.0,0.0,A.1,164.077881,1.5,0.997563,0,0,0,0.000000e+00,0.000000
